IPython Parallel must be installed before it can be used. On the Bash shell, in your preferred conda environment, run these two commands:

pip install ipyparallel

for now, put only a fraction of the number of cores you have in your machine into the cluster
if you have at least 8 cores in your machine, then 4 should be fine to start with.

ipcluster start -n 4 --location=localhost

By default, ipcluster will start a local cluster on your own computer, utilizing your CPU's multiple cores if available.

Once it has started, launch jupter notebook.

Next, we open a notebook and import Client from ipyparallel

In [13]:
from ipyparallel import Client
import os

home_dir = home_dir = os.path.expanduser('~')

In [14]:
with open(home_dir + '/datafile/reformatted_Unigene.fa') as inFile:
    LoS = inFile.read().splitlines()
    LoL = []
    for record in LoS:
        LoL.append(record.split())

In [10]:
rc = Client()
dv = rc[:]

# the parallelized function,
# which renders the reverse complement
# from each record's data string,
# and appends it to the end of the record
@dv.parallel(block = True)
def process_LoL(LoL):
    def revCompl(nuclStr):
        outputStr, complDict =\
        '', {'A':'T', 'C':'G', 'G':'C', 'T':'A', 'N':'N'}
        for nucleotide in nuclStr:
            outputStr = complDict[nucleotide]\
                    + outputStr
        return outputStr
        
    for record in LoL:
        revc = revCompl(record[6])
        record.append(revc)
    return LoL                


results = process_LoL(LoL)

In [11]:
# to see all the instances of python running in parallel
# first open a terminal window and run: top
# watch top after you start the profiling statement below:
%timeit process_LoL(LoL)

677 ms ± 41.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
